<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

## Step 2 - Loading Data & Preprocessing

In [3]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [4]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)

Found 29031 files belonging to 7 classes.
Found 7378 files belonging to 7 classes.


In [5]:
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

In [6]:
train_batch = train_data_iterator.next()
test_batch = test_data_iterator.next()

In [7]:
test_batch

(array([[[[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [254., 254., 254.],
          [254., 254., 254.],
          [254., 254., 254.]],
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [254., 254., 254.],
          [254., 254., 254.],
          [254., 254., 254.]],
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [254., 254., 254.],
          [254., 254., 254.],
          [254., 254., 254.]],
 
         ...,
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.]],
 
         [[ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          [ 34.,  34.,  34.],
          ...,
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.],
          [ 48.,  48.,  48.

In [8]:
len(test_batch)

2

In [9]:
train_batch[0].shape

(32, 256, 256, 3)

In [10]:
train_data = train_data.map(lambda x,y: (x/255,y))
test_data = test_data.map(lambda x,y: (x/255,y))

In [11]:
scaled_train_iterator = train_data.as_numpy_iterator()
scaled_test_iterator = test_data.as_numpy_iterator()
train_batch = scaled_train_iterator.next()
test_batch = scaled_test_iterator.next()
test_batch[0].max()

1.0

In [12]:
len(train_data)

908

In [54]:
train_size = int(len(train_data)*0.7)
val_size = int(len(train_data)*0.3)+1
test_size = len(test_data)
train_size+val_size

908

In [14]:
train = train_data.take(train_size)
val = train_data.skip(train_size).take(val_size)
test = test_data.take(test_size)

In [15]:
print(len(train),len(val),len(test))

726 182 231


## Step 3 - Model

In [62]:
model = Sequential(
    [
        Conv2D(16, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),

        Conv2D(32, (3,3), 1,  activation = 'relu'),
        MaxPooling2D(),

        Conv2D(16, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),

        Flatten(),

        Dense(256, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [63]:
model.compile(
    optimizer = 'adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [64]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 127, 127, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 123008)            0         
                                                                 
 dense_10 (Dense)            (None, 256)              

In [65]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [66]:
history = model.fit(train, validation_data = val, epochs = 20, callbacks = [tensorboard_callback])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


726/726 [==============================] - 101s 136ms/step - loss: 1.8005 - accuracy: 0.3265 - val_loss: 1.6615 - val_accuracy: 0.3633
Epoch 2/20
726/726 [==============================] - 100s 137ms/step - loss: 1.5344 - accuracy: 0.4048 - val_loss: 1.5383 - val_accuracy: 0.3973
Epoch 3/20
726/726 [==============================] - 104s 143ms/step - loss: 1.3739 - accuracy: 0.4748 - val_loss: 1.5507 - val_accuracy: 0.4104
Epoch 4/20
726/726 [==============================] - 111s 153ms/step - loss: 1.1430 - accuracy: 0.5692 - val_loss: 1.6026 - val_accuracy: 0.4251
Epoch 5/20
726/726 [==============================] - 134s 185ms/step - loss: 0.8266 - accuracy: 0.6987 - val_loss: 1.9597 - val_accuracy: 0.4170
Epoch 6/20
726/726 [==============================] - 99s 135ms/step - loss: 0.5132 - accuracy: 0.8211 - val_loss: 2.3424 - val_accuracy: 0.4316
Epoch 7/20
726/726 [==============================] - 97s 133ms/step - loss: 0.3003 - accuracy: 0.9007 - val_loss: 2.9262 - val_accuracy

In [67]:
model.save('/content/gdrive/MyDrive/Human emotions/Saved_model')
